# 基本操作

In [1]:
# 使用%matplotlib命令可以将matplotlib的图表直接嵌入到Notebook之中
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

In [2]:
# 加载mnist数据
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("data/MNIST", one_hot=True)

Extracting data/MNIST\train-images-idx3-ubyte.gz
Extracting data/MNIST\train-labels-idx1-ubyte.gz
Extracting data/MNIST\t10k-images-idx3-ubyte.gz
Extracting data/MNIST\t10k-labels-idx1-ubyte.gz


数据集分三大部分，（训练数据，测试数据和校验数据）
暂时不知道校验数据的作用

In [3]:
print("Training-set:{}".format(len(data.train.labels)))
print("Test-set:{}".format(len(data.test.labels)))
print("Validation-set:{}".format(len(data.validation.labels)))

Training-set:55000
Test-set:10000
Validation-set:5000


## One-Hot 编码
labels将由原本的一个数转换成一个n * 1的向量，n为所有可能的输出的数目

查看test集前5个的label的One-Hot编码

In [4]:
data.test.labels[0:5]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

某些情况需要把One-Hot编码的向量转换成数值型的数据，
可以实现转换的方法是从向量中找出数值最大的，也就是1，
取其下标作为这个向量对应的数据型数据。

看到这里，有点像之前看过的遗传算法的编码和解码

In [5]:
arr = [
    [1,2,3,4],
    [3,4,5,2],
]
b = np.array(arr)
# print(b)
print(type(b))
print(b)
print(type(data.test.labels[0]))
print(b.argmax())
# 由以上代码可知
# np.array接收一个list，并转换成numpy里的ndarray对象
# 测试数据集每个label的类型也是ndarray
# ndarray的argmax就是找出比较ndarray里面的每一个元素，找到最大的返回其下标
# 从我的例子里面看，即使是二维数组，但也只是返回一个数字，而不是返回横纵两个坐标的参数

<class 'numpy.ndarray'>
[[1 2 3 4]
 [3 4 5 2]]
<class 'numpy.ndarray'>
6


In [6]:
data.test.cls = np.array([label.argmax() for label in data.test.labels])
data.test.cls[0:5]

array([7, 2, 1, 0, 4], dtype=int64)

## tensorflow的数据类型
[链接](http://wiki.jikexueyuan.com/project/tensorflow-zh/get_started/basic_usage.html)

- 使用**图(graph)**来表示计算任务
- 在**会话(Session)**的上下文(context)中执行图
- 使用tensor表示数据
- 通过变量(Variable)维护状态
- 使用feed和fecth可以为任意的操作赋值或者从其中获取数据

看个矩阵相乘的例子

构建图

In [7]:
# create one constant op resprents a 1x2 matrix
# it is loaded into the default graph
matrix1 = tf.constant([[3., 3.]])

# other constant op, 2x1 matrix
matrix2 = tf.constant([[2.],[2.]])
print(matrix1.shape)
print(matrix2.shape)
# create a matrix multi~ op, 
result = tf.matmul(matrix1, matrix2)
print(result)

(1, 2)
(2, 1)
Tensor("MatMul:0", shape=(1, 1), dtype=float32)


在会话中启动图

In [8]:
# start default graph
sess = tf.Session()

# invoke method 'run' of sess to execute the multi~ op
# 'result' represent the output of the multi op
# so now, regard 'result' as input to the function 'run'

graph_result = sess.run(result)
print(graph_result)

# close session
sess.close()

[[ 12.]]


In [9]:
# simply code
with tf.Session() as sess:
    r = sess.run(result)
    print(r)

[[ 12.]]


## Tensorflow Graph
Tensorflow使用流程，构建图然后去计算图，得到图的输出

计算图(computational graph)是比直接在Python中进行计算要高效的多，
除此之外，Tensorflow能自动计算需要优化的变量的梯度

一个TensorFlow图由下面几个部分组成:
- **Placeholder** 变量 -- 用来改变图的输入
- **Model** 变量 -- Model变量会自动优化该变量的梯度（我猜的）
像线性模型里面的W和b
- 模型本质上就是数学函数，接受Placeholder变量和Model变量计算一些输出
- A **cost measure** is used to  guide the optimization of the variables
- An **optimization method** -- update the variables of the model.

### Placeholder variables
Placeholder variables as the input of the graph.
it may change each time when execute the graph.


In [11]:
# set datatype as float32, shape as [row, col] None represented unset
x = tf.placeholder(tf.float32, [None, 3])
print(x)

Tensor("Placeholder:0", shape=(?, 3), dtype=float32)


# 画图

In [ ]:
fig, axes = plt.subplots(3,3)

for i ,ax in enumerate(axes.flat):
    print(ax)
    ax.set_xlabel(i)
    
    # remove ticks
    ax.set_xticks([])
    ax.set_yticks([])
# 可以看出axes.flat包括了所有的子图
# 输出子图信息可得该子图某个点（暂时不知道是哪个点）和形状（长x宽）

In [ ]:
def watch(images, cls_sample ):
    """
    @param 
    """
    assert len(images) == len(cls_sample)
    
    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(3,3)
    
    for i, ax in enumerate(axes.flat):
        # Plot image
        ax.imshow(images[i].reshape((28 , 28)), cmap='binary')
        
        xlabel = "True: {0}".format(cls_sample[i])
        
        ax.set_xlabel(xlabel)
        
        # remove ticks
        ax.set_xticks([])
        ax.set_yticks([])

In [ ]:
watch(data.test.images[0:9], data.test.cls[0:9])